<div style='width=100%; display:flex;flex-direction:row'><img  src=https://universidadeuropea.com/resources/media/images/universidad-europea-logo_poc9mEM.original.png width=100  style='  margin-left: auto;margin-right: auto; width: 25%; height:25%;'><img  src=https://i.ibb.co/1068C7j/EATEASER.jpg width=100 style='  margin-left: auto;margin-right: auto; width: 10%;height:25%;'></div>

<div style='margin:auto;text-align: center;font-family: "Times New Roman", Times, serif; font-weight: bold;'>PROYECTO COMUTACIONAL<br><br>EATEASER - VOZ A TEXTO</div>

<div style='width:100%; display:flex;flex-direction:row'>
    <div style='width:50%;margin-right:5cm;'>
        <p style='font-family: "Times New Roman", Times, serif; font-weight: bold;'>ESTUDIANTES</p>
<ul style='font-family: "Times New Roman", Times, serif;'>
    <li>Adilem Dobras 21911633</li><li>Roberto Echevarria 21823680</li><li>Carlos Gonzales 22067726</li><li>Juan Carlos Rondeau 21816176</li></ul> </div>

</div>

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">1. Importamos las librerias</h2>

In [1]:
"""
!pip install pytube
!pip install SpeechRecognition
!pip install pydub
!pip install moviepy
!pip install beautifulsoup4
!pip install nltk
"""

'\n!pip install pytube\n!pip install SpeechRecognition\n!pip install pydub\n!pip install moviepy\n!pip install beautifulsoup4\n!pip install nltk\n'

In [2]:
from pytube import YouTube
import speech_recognition as sr
from pydub import AudioSegment
import moviepy.editor as mp
from bs4 import BeautifulSoup
import glob
import os
import re
from os import listdir
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import requests
from pydub.silence import split_on_silence
import pathlib

C:\Users\Carlow\.conda\envs\EatEaser\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">2. Inicio del programa</h2>

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE CONTROLADORVIDEO</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará los ajustes para manejar el video recibido y manipularlo.</p>


In [3]:
class ControladorVideo:
    def __init__(self,enlace): 
        self._idvideo = self.reenumerar()
        self.enlacevideo=enlace
        self.yt=YouTube(self.enlacevideo)
        self.nombrevideo=''
        self.titulovideo=self.yt.title
        self.autorvideo=self.yt.author
        self.fechavideo=self.yt.publish_date
        self.duracionvideo=self.yt.length
    """|DESCARGAR VIDEO URL: descarga el video de youtube
       |return: devuelve una ruta absoluta"""
    def descargarVideoURL(self):
        recetasVideos = 'recetasvideos/'
        #aqui creo un nuevo id para el nuevo video
        self._idvideo= self._idvideo+1
        #esta sera el archivo del video y su nuevo nombre
        nombre='receta'+str(self._idvideo)
        #le pedimos al pytube que solo nos descargue el audio y lo descargamos
        t=self.yt.streams.filter(file_extension='mp4').first().download(output_path=recetasVideos,filename=nombre+'.mp4')
        #devolvemos el nombre
        return nombre
    """|PARSEO VIDEO: pasa el video de .mp4 a .wav
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve el nuevo nombre del audio en .wav"""
    def parseoVideo(self,nombre):
        recetasVideos = 'recetasvideos/'
        #tomamos el video en mp4 
        track = mp.VideoFileClip(recetasVideos+nombre+'.mp4')
        #cambiamos el video a .wav
        nombre_wav="{}.wav".format(nombre)
        track.audio.write_audiofile(recetasVideos+nombre_wav)
        track.close()
        return nombre+'.wav'
    def reenumerar(self):
        contador = 0
        for path in pathlib.Path("./recetasvideos").iterdir():
            if path.is_file():
                contador += 1
        return contador
    """|SPEECH TEXT:Transforma el audio a texto
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve un string con el texto devuelto"""
    def speech_text(self,nombre):
        recetasVideos = 'recetasvideos/'
        #instanciamos el recognizer
        r = sr.Recognizer()
        audio = sr.AudioFile(recetasVideos+nombre)
        with audio as source:
            audio_file = r.record(source)
        #transcribimos el audio a texto
        result = r.recognize_google(audio_file, language = 'es-ES')
        return result
        

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE DEPURADOR</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará el proceso de extraccion, transformacion y carga de nuestro programa EATEASER.</p>


In [4]:
#si el video es mayor de 3 minutos no funciona
#si el video es en ingles no funciona
class Depurador:
    def __init__(self): 
        self.rec=RecursosAdicionales()
    """|EXTRACCION: proceso etl donde extraemos al informacion del video 
       |enlace: es un string que se colocara el enlace del video"""
    def extraccion(self,enlace):
        #instanciamos el controlador de videos
        cv=ControladorVideo(enlace)
        #paso 1: descargamos el video
        cv.nombrevideo=cv.descargarVideoURL()
        #paso 2: pasamos el video a .wav
        nombre=cv.parseoVideo(cv.nombrevideo)
        #paso 3: pasamos el .wav a texto
        result=cv.speech_text(nombre)
        #paso 4: escribimos el texto recibido en un txt
        resultado=self.rec.escritura(cv.nombrevideo,"Titulo:"+cv.titulovideo+"\n"+"Autor:"+cv.autorvideo+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
        #paso 5: eliminamos los mp4
        self.rec.eliminacionMp4()
        return resultado     
    def transformacion(self):
        print()
    def carga(self):
        print()

In [5]:
#aqui iran lecturas-escrituras-guardar-eliminar cosas en bases de datos
class RecursosAdicionales:
    """|ESCRITURA: escribe textos txt
       |nombre: nombre del 
       |return: devuelve el audio en texto"""
    #yo siento que en vez de cambiar de carpeta los textos...deberiamos eliminar los mp4
    #y si lo mas factible aqui es que guardemos el video por su nombre?
    #creo que no debemos contar los files porque cansan el programa
    
    def escritura(self,nombre,texto):
        recetasTextos = './recetastextos/'
        if not(os.path.exists(recetasTextos)):
            os.mkdir(recetasTextos)
        f = open(recetasTextos+nombre+'.txt', 'w')
        f.write(texto)
        f = open(recetasTextos+nombre+'.txt', "r")
        print(f.read())
        f.close()
    def eliminacionMp4(self):
        recetasVideos = './recetasvideos/'
        py_files = glob.glob(recetasVideos+'*.mp4')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}")      
        

In [6]:
class WebScrap:
    def __init__(self): 
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    def request(self, url):
        request1 = requests.get(url, headers=self.headers)
        html = request1.content
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    def verificar_alimento(self,alimento):
        soup = self.request( 'https://www.themealdb.com/api/json/v1/1/search.php?s='+alimento)
        print(soup)
        

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">3. Main</h2>

In [7]:
#y si tambien vemos si le permitimos al usuario que meta videos?
dep=Depurador()
if __name__ == '__main__':
    #dep.extraccion('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.extraccion('https://www.youtube.com/watch?v=PsqR5M8rdjA&list=LL&index=9&t=4s')
    #dep.extraccion('https://www.youtube.com/watch?v=xfYcM_jHgPY')
    #dep.extraccion('https://www.youtube.com/watch?v=wiCfqc5W-yo')
    #error#dep.extraccion('https://www.youtube.com/watch?v=3DnPkf9rP_0')
    #error#dep.extraccion('https://www.youtube.com/watch?v=xVsgKMZFCZY')
    dep.extraccion('https://www.youtube.com/watch?v=rpCe0RPMY94')
    dep.extraccion('https://www.youtube.com/watch?v=rv4gLMa-FYk')
    dep.extraccion('https://www.youtube.com/watch?v=VS8zYxBj4r8')
    dep.extraccion('https://www.youtube.com/watch?v=o99JXrEkZoo')
    dep.extraccion('https://www.youtube.com/watch?v=lKkg5L23b3M')
    dep.extraccion('https://www.youtube.com/watch?v=PsqR5M8rdjA&t=14s')
    dep.extraccion('https://www.youtube.com/watch?v=IvZaAL6qYe0&t=29s')
    dep.extraccion('https://www.youtube.com/watch?v=SIMQBuuyE9M')
    dep.extraccion('https://www.youtube.com/watch?v=_YoZfg7R8Hk')
    dep.extraccion('https://www.youtube.com/watch?v=Zv7KdlOBk7Y')
    dep.extraccion('https://www.youtube.com/watch?v=mFcN4btaZyI&t=2s')
    
    
    

MoviePy - Writing audio in recetasvideos/receta5.wav


MoviePy - Done.
Titulo:¿Es esta la mejor receta de Pasta Italiana? Increíble sabor | Fetuccini con Salsa Alfredo
Autor:Platos Fáciles con Tamara
Fecha Publicacion:2022-03-06 00:00:00
Entradilla:hola chicos bienvenidos a otro día más a mi canal hoy es traigo una receta de pasta deliciosa con salsa Alfredo no os la perdáis porque vais a ver qué es muy fácil de preparar empezamos poniendo una cazuela con abundante agua fuego alto cuando el agua comience a hervir echamos la sal e incorporamos la pasta en este caso estoy utilizando fetuccini que suele llevar la pasta con salsa Alfredo pero se puede sustituir por otras clases de pasta ya sea larga o corta cuando hayamos añadido la pasta y el agua arranque a hervir de nuevo empezamos a contar los minutos y cocinamos el tiempo que recomienda el fabricante para que quede al dente pasos este tiempo retiramos la cazuela del fuego antes de escurrir la pasta reservamos un poco del agua de cocción por si hubiera que aligerar la salsa más adelante ah

MoviePy - Done.
Titulo:Patatas Mayordomo | Si las pruebas no querrás prepararlas de otra manera
Autor:Platos Fáciles con Tamara
Fecha Publicacion:2022-01-30 00:00:00
Entradilla:hola chicos bienvenidos otro día más a mi canal hoy vamos a preparar unas deliciosas y facilísimas patatas mayordomo lo primero que vamos a hacer es pelar las patatas las lavamos y las cortamos en rodajas gruesas pero que no pasen el medio centímetro de grosor también cortamos las cebollas en juliana a continuación pasamos a pochar la cebolla para hacerlo echamos la mantequilla en una cacerola baja y la ponemos a calentar a fuego entre medio y alto la removemos y una vez que esté completamente derretida echamos la cebolla y la rehogamos durante unos minutos cuando esté blandita y ligeramente doradita añadimos las patatas en este momento aprovechamos para sazonar las con sal al gusto un poco de pimienta recién molida y perejil picado si podéis utilizar fresco mucho mejor revolvemos muy bien para que todos los ing

MoviePy - Done.
Titulo:SANGRIA - Deliciosa y Refrescante al Estilo Tanto Monta
Autor:Tanto Monta En La Cocina
Fecha Publicacion:2021-08-03 00:00:00
Entradilla:con este calor tan increíble que está haciendo usted antoja algo rico y refrescante por tal motivo el día de hoy te voy a enseñar a hacer sangría pero no cualquiera se cría y hablamos un poquito y la vamos a hacer al estilo de tanto monta tienes novio el canal te pido suscríbete al mismo y no se te olvide que te bajito de la descripción del vídeo están todos los ingredientes mi nombre Sandra y una vez más te doy la bienvenida tanto monta en la cocina para esta receta vamos a necesitar una taza de jugo de naranja de preferencia natural es súper superfácil esta receta y deveras te queda exquisita en una jarra lo suficientemente grande que lo que cueste limpia vamos a poner nuestra casa y te jugo de naranja vamos a necesitar también una larga que lo que con la tenemos bien lavada y lo único que tenemos que hacer es cortarla en reban

MoviePy - Done.
Titulo:Jugo de Apio y Limon / EN AYUNAS
Autor:Tanto Monta En La Cocina
Fecha Publicacion:2020-09-08 00:00:00
Entradilla:hacer un vídeo muy cortito muy interesante vamos a hacer un cubo de apio con limón al igual que siempre vas a ver qué cosa tan rica y fácil de preparar como todos sabemos en la que tiene muchos beneficios entre ellos tiene muchísimos minerales alcalinos tiene vitaminas A B1 B2 B6 vitamina C es bueno para la hipertensión ayuda un poquito para bajar de peso te sirve para el estreñimiento por su contenido en fibra si sirve también como diurético en fin son muchos los beneficios yo no sé nuestro bloque ni pretendo serlo simplemente quiero compartir contigo y está receta que una nutriologa pues que me da mi nombre es Sandra y una vez más te doy la bienvenida tanto monta en la cocina donde quiero compartir contigo recetas fáciles sabores nuevos problemas importantes para compartirlo con toda la familia por esta receta vamos a utilizar 4 tallos de apio ya los

...

MoviePy - Done.
Titulo:Magdalenas de calabaza saludables y sin azúcar
Autor:Avena y Aguacate
Fecha Publicacion:2019-10-19 00:00:00
Entradilla:hola a todos y bienvenidos a un nuevo vídeo de avena y aguacate hoy volvemos a preparar juntos un desayuno saludable magdalenas de calabaza el otoño es la temporada desde vegetal tan nutritivo rico en fibra potasio y vitaminas A y B por lo que es bueno para la piel la vista y el sistema cardiovascular así que si queréis aprender a preparar estas magdalenas tan saludables que hagas un ratito conmigo ahora vamos a mear las magdalenas a 180 grados durante 20 o 25 minutos fáciles preparar estás magdalenas de calabaza que están riquísimas y son perfectas para desayunos o meriendas tenéis que probarlas porque sé que os van a encantar si él no me seguís en Instagram y estáis suscritos a mi blog tenéis los enlaces justo debajo del vídeo allí encontrareis todas mis recetas y un montón de contenido para inspiraros a llevar una vida más saludable y suscribi

MoviePy - Done.
Titulo:Bombones de avellana y chocolate | Ferrero Rocher saludables | Trufas sabor Nutella sin azúcar
Autor:Avena y Aguacate
Fecha Publicacion:2019-10-12 00:00:00
Entradilla:hola a todos y bienvenidos a una nueva receta de avena y aguacate hoy os traigo un dulce saludable con el que sorprender a invitados o también para picar cuando os apetezca un capricho de lo más sano bombones de cacao y avellanas como todos los frutos secos son fuente de grasas saludables que nos protegen frente a enfermedades además de ser ricas en antioxidantes y minerales centros de avellanas más grandes así daremos un toque crujiente vamos a dejar enfriar los bombones en la nevera aproximadamente media hora si ya hemos estado estos son buenas dale me gusta y suscríbete a mi canal para no perderte ninguna de mis recetas saludables hasta la
MoviePy - Writing audio in recetasvideos/receta13.wav


MoviePy - Done.
Titulo:Scones de limón con harina integral | Receta fácil
Autor:Avena y Aguacate
Fecha Publicacion:2019-09-21 00:00:00
Entradilla:hola a todos y bienvenidos a una nueva receta dulce y saludable de avena y aguacate hoy vamos a preparar escols de limón un dulce de origen inglés pero vamos a hacerlo en su versión más saludable integral con avena y sin nada de azúcar los scouts son muy sencillos pero para conseguir su inconfundible textura la receta tiene unas particularidades técnicas que vamos a ver paso a paso para preparar estos auténticos es importante usar mantequilla muy fría no le la mantequilla es una grasa láctea de buena calidad incorpora los trozos de mantequilla con un tenedor hasta que quede una textura como de migas de pan vamos a mear los escobas a 180 grados entre 12 y 14 minutos hasta que los bordes epicenas tarde heladitos si es así dale a Me Gusta y no os olvidéis de suscribiros para seguir viendo recetas tan saludables como riquísimas hasta la próxima
M

MoviePy - Done.
Titulo:Brownie saludable de avena y aguacate | Receta fit sin gluten y sin azúcar
Autor:Avena y Aguacate
Fecha Publicacion:2019-09-08 00:00:00
Entradilla:hola a todos y bienvenidos a una receta más de avena y aguacate y esta vez de verdad porque por primera vez vamos a preparar una receta con avena y aguacate como ingredientes aguacate que son muy saludables y nutritivos sin azúcar y aptos para celíacos se utilizan avena certificada sin gluten este brownie de avena y aguacate triunfará hasta entre los más golosos y pueden disfrutar de él tanto adultos como niños si os ha gustado está receta dame like y si queréis seguir recibiendo recetas saludables tan ricas como está suscribiros a mi canal hasta la presión
MoviePy - Writing audio in recetasvideos/receta15.wav


MoviePy - Done.
Titulo:Galletas de mantequilla de cacahuete saludables y veganas | Receta sana sin azúcar
Autor:Avena y Aguacate
Fecha Publicacion:2019-08-09 00:00:00
Entradilla:bienvenidas a través de receta de avena y aguacate hoy vamos a preparar unas galletas veganas de crema de cacahuete que también son aptas para alérgicos a los lácteos y al huevo y como siempre son muy saludables integrales y sin azúcar también puedes ayudarte con las manos ya tenemos la masa y ahora vamos a dejarla enfriar como hora y media a 2 horas en la nevera o unos 30 o 40 minutos en el congelador las galletas unos 10 minutos a 180 grados es importante que las dejes con cuidado en la rejilla para que se enfríen y se endurezca estás galletas y los riquísimas que están son perfectas para una merienda saludable o para un tentempié de media mañana
